In [ ]:
import torch
import transformers

print("--- 环境诊断开始 ---")

# 1. 检查 PyTorch 版本
print(f"PyTorch 版本: {torch.__version__}")

# 2. 检查 Transformers 版本
print(f"Transformers 版本: {transformers.__version__}")

# 3. 核心测试：检查GPU是否可用
is_cuda_available = torch.cuda.is_available()
print(f"\nGPU 是否可用: {is_cuda_available}")

if is_cuda_available:
    # 如果GPU可用，打印更多信息
    gpu_count = torch.cuda.device_count()
    current_gpu_index = torch.cuda.current_device()
    gpu_name = torch.cuda.get_device_name(current_gpu_index)
    
    print(f"检测到 {gpu_count} 个GPU。")
    print(f"当前使用的GPU索引: {current_gpu_index}")
    print(f"当前GPU名称: {gpu_name}")
    
    # 简单的张量运算测试，确认GPU可以工作
    try:
        x = torch.tensor([1.0, 2.0, 3.0]).to("cuda")
        print(f"成功将张量移动到GPU: {x}")
        print("GPU工作正常！")
    except Exception as e:
        print(f"GPU运算时出错: {e}")

else:
    print("未检测到可用的NVIDIA GPU。模型将会在CPU上运行。")
    print("（注意：在CPU上运行大模型会非常慢。）")

print("\n--- 环境诊断结束 ---")

In [ ]:
import torch
from transformers import pipeline

# --- 1. 定义模型ID和加载配置 ---
# 我们选择Qwen1.5的7B对话模型
model_id = "Qwen/Qwen1.5-7B-Chat"

print(f"正在加载模型: {model_id}")
print("这将在第一次运行时下载模型文件（约15GB），请耐心等待...")

# 创建文本生成管道
# device_map="auto" 会自动将模型加载到您的RTX 4060 Ti上
# torch_dtype="auto" 会为您的显卡自动选择最佳的数据类型（bfloat16）
# load_in_4bit=True 会启用4-bit量化，极大降低显存占用
pipe = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
    torch_dtype="auto",
    model_kwargs={"load_in_4bit": True}
)

print("模型加载完毕，准备生成文本！")

# --- 2. 创建对话内容 ---
# 对于对话模型，我们需要遵循它指定的对话格式
# 这里我们创建一个包含系统指令和用户问题的消息列表
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "你好！请你用中文介绍一下自己，并写一首关于夏天的五言绝句。"}
]

# 使用分词器（tokenizer）的应用聊天模板功能，将消息列表转换为模型能理解的单个字符串
# 这是与对话模型交互的推荐做法
prompt = pipe.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)


# --- 3. 生成并打印回复 ---
# do_sample=True 允许使用更多样化的生成策略
# temperature 和 top_p 用于控制生成文本的创造性和随机性
# max_new_tokens 控制模型最多生成多少个新词
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7,
    top_p=0.95
)

# outputs[0]["generated_text"] 包含了完整的对话（包括我们的输入）
# 我们只关心模型生成的部分，所以需要切片掉输入的部分
response = outputs[0]["generated_text"][len(prompt):]

print("\n--- AI 的回复 ---")
print(response)

In [1]:
from transformers import pipeline
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


In [87]:
classifier(["","123","1234","china","Iraq"])

[{'label': 'POSITIVE', 'score': 0.7481212615966797},
 {'label': 'POSITIVE', 'score': 0.9943673014640808},
 {'label': 'POSITIVE', 'score': 0.9821881651878357},
 {'label': 'POSITIVE', 'score': 0.9924151301383972},
 {'label': 'NEGATIVE', 'score': 0.9922665953636169}]

In [88]:
from datasets import load_dataset, Audio